In [4]:
import torch
import os
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, DefaultDataCollator, Trainer, TrainingArguments, BertTokenizer
from datasets import Dataset, load_dataset, DatasetDict
import json
import pandas as pd


In [10]:
# Load the model from a .pt file
model_add = "C:\\Users\\CShuwen\\Desktop\\nlp_Project\\CS4248_G10\\bert_huggingface_model\\bert_baseline.pt"
model = torch.load(model_add, map_location=torch.device('cpu'))
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Ensure the model is in evaluation mode (not training mode)
model.eval()


BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elem

In [11]:
CWD = os.getcwd()

parent_dir = os.path.dirname(CWD)

parent_dir = parent_dir.replace("\\", "\\\\")


class preprocess_huggingface():
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

        self.train_squad = open_file("C:\\Users\\CShuwen\\Desktop\\nlp_Project\\CS4248_G10\\dataset\\train-v1.1.json")
        # self.train_data = dataset_parse(self.train_squad).train_test_split(test_size=0.2)
        self.train_data = dataset_parse(self.train_squad)

        self.dev_squad = open_file("C:\\Users\\CShuwen\\Desktop\\nlp_Project\\CS4248_G10\\dataset\\dev-v1.1.json")
        self.dev_data = dataset_parse(self.dev_squad)

        self.data = DatasetDict({"train": self.train_data, "validation": self.dev_data})


    def prepare_train_features(self, examples):
        # Some of the questions have lots of whitespace on the left, which is not useful and will make the
        # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
        # left whitespace
        examples["question"] = [q.lstrip() for q in examples["question"]]

        # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
        # in one example possible giving several features when a context is long, each of those features having a
        # context that overlaps a bit the context of the previous feature.
        tokenized_examples = self.tokenizer(
            examples["question"],
            examples["context"],
            truncation="only_second",
            max_length=384,
            stride=128,
            return_overflowing_tokens=True,
            return_offsets_mapping=True,
            padding="max_length",
        )

        # Since one example might give us several features if it has a long context, we need a map from a feature to
        # its corresponding example. This key gives us just that.
        sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
        # The offset mappings will give us a map from token to character position in the original context. This will
        # help us compute the start_positions and end_positions.
        offset_mapping = tokenized_examples.pop("offset_mapping")

        # Let's label those examples!
        tokenized_examples["start_positions"] = []
        tokenized_examples["end_positions"] = []

        for i, offsets in enumerate(offset_mapping):
            # We will label impossible answers with the index of the CLS token.
            input_ids = tokenized_examples["input_ids"][i]
            cls_index = input_ids.index(self.tokenizer.cls_token_id)

            # Grab the sequence corresponding to that example (to know what is the context and what is the question).
            sequence_ids = tokenized_examples.sequence_ids(i)

            # One example can give several spans, this is the index of the example containing this span of text.
            sample_index = sample_mapping[i]
            answers = examples["answers"][sample_index]
            # If no answers are given, set the cls_index as answer.
            if len(answers["answer_start"]) == 0:
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Start/end character index of the answer in the text.
                start_char = answers["answer_start"][0]
                end_char = start_char + len(answers["text"][0])

                # Start token index of the current span in the text.
                token_start_index = 0
                while sequence_ids[token_start_index] != 1:
                    token_start_index += 1

                # End token index of the current span in the text.
                token_end_index = len(input_ids) - 1
                while sequence_ids[token_end_index] != 1:
                    token_end_index -= 1

                # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
                if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                    tokenized_examples["start_positions"].append(cls_index)
                    tokenized_examples["end_positions"].append(cls_index)
                else:
                    # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                    # Note: we could go after the last offset if the answer is the last word (edge case).
                    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                        token_start_index += 1
                    tokenized_examples["start_positions"].append(token_start_index - 1)
                    while offsets[token_end_index][1] >= end_char:
                        token_end_index -= 1
                    tokenized_examples["end_positions"].append(token_end_index + 1)

        return tokenized_examples

    def prepare_validation_features(self, examples):
        # Some of the questions have lots of whitespace on the left, which is not useful and will make the
        # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
        # left whitespace
        examples["question"] = [q.lstrip() for q in examples["question"]]

        # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
        # in one example possible giving several features when a context is long, each of those features having a
        # context that overlaps a bit the context of the previous feature.
        tokenized_examples = self.tokenizer(
            examples["question"],
            examples["context"],
            truncation="only_second",
            max_length=384,
            stride=128,
            return_overflowing_tokens=True,
            return_offsets_mapping=True,
            padding="max_length",
        )

        # Since one example might give us several features if it has a long context, we need a map from a feature to
        # its corresponding example. This key gives us just that.
        sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

        # We keep the example_id that gave us this feature and we will store the offset mappings.
        tokenized_examples["example_id"] = []

        for i in range(len(tokenized_examples["input_ids"])):
            # Grab the sequence corresponding to that example (to know what is the context and what is the question).
            sequence_ids = tokenized_examples.sequence_ids(i)
            context_index = 1

            # One example can give several spans, this is the index of the example containing this span of text.
            sample_index = sample_mapping[i]
            tokenized_examples["example_id"].append(examples["id"][sample_index])

            # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
            # position is part of the context or not.
            tokenized_examples["offset_mapping"][i] = [
                (o if sequence_ids[k] == context_index else None)
                for k, o in enumerate(tokenized_examples["offset_mapping"][i])
            ]

        return tokenized_examples
    def get_train_set(self):
        ds = self.data
        tokenized = ds.map(self.prepare_train_features, batched=True, remove_columns=ds["train"].column_names)

        return tokenized
    
    def get_test_set(self):
        ds = self.data
        validation_features = ds.map(
            self.prepare_validation_features,
            batched=True,
            remove_columns=ds["validation"].column_names
        )

        return validation_features
    
    def get_data(self):
        return self.data

def open_file(file_path):
    try:
        print(file_path)
        f = open(file_path)
        dataset = json.load(f, strict=False)["data"]
        return dataset
    except FileNotFoundError:
        print("File not found.")
    except Exception as e:
        print(f"An error occurred: {str(e)}")

def dataset_parse(dataset):
    pre_dataframe = []
    for data in dataset:
        for p in data["paragraphs"]:
            for pqas in p["qas"]:
                for ans in pqas["answers"]:
                    pre_dataframe.append([pqas["id"], data["title"], p["context"], pqas["question"].strip(), { "answer_start": [int(ans["answer_start"])], "text": ans["text"] }])

    df = pd.DataFrame(pre_dataframe, columns=["id", "title", "context", "question", "answers"])
    ds = Dataset.from_pandas(df)

    return ds

In [12]:
data_collator = DefaultDataCollator()

training_args = TrainingArguments(
    output_dir="../tmp/bert-squad-baseline-model",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    evaluation_strategy="epoch",
    weight_decay=0.01,
    logging_strategy="epoch",
    save_strategy="epoch",
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    tokenizer=tokenizer,
)


In [ ]:
bert_squad = preprocess_huggingface(tokenizer)
# extract the test data set
validation_set = bert_squad.get_test_set()['validation']
# train_set = bert_squad.get_train_set()['train']

C:\Users\CShuwen\Desktop\nlp_Project\CS4248_G10\dataset\train-v1.1.json
C:\Users\CShuwen\Desktop\nlp_Project\CS4248_G10\dataset\dev-v1.1.json


Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

In [ ]:
validation_set.set_format(type=validation_set.format["type"], columns=list(validation_set.features.keys()))
predictions = trainer.predict(validation_set)
print(predictions.predictions[1])
print(predictions)

In [9]:
# printed using slurm:
"""
Map: 100%|██████████| 87599/87599 [00:40<00:00, 2152.79 examples/s]
Map: 100%|██████████| 34726/34726 [00:16<00:00, 2115.27 examples/s]
100%|██████████| 4341/4341 [20:50<00:00,  3.47it/s]
[[ -6.7124968  -9.547691   -9.648446  ... -10.148347  -10.147022
  -10.147211 ]
 [ -6.7124968  -9.547691   -9.648446  ... -10.148347  -10.147022
  -10.147211 ]
 [ -6.7124968  -9.547691   -9.648446  ... -10.148347  -10.147022
  -10.147211 ]
 ...
 [ -7.3926897  -9.895513  -10.14147   ... -10.213227  -10.218759
  -10.222772 ]
 [ -7.3926897  -9.895513  -10.14147   ... -10.213227  -10.218759
  -10.222772 ]
 [ -7.3926897  -9.895513  -10.14147   ... -10.213227  -10.218759
  -10.222772 ]]
PredictionOutput(predictions=(array([[ -6.7151318,  -9.602398 ,  -9.5187435, ..., -10.077262 ,
        -10.07667  , -10.0787945],
       [ -6.7151318,  -9.602398 ,  -9.5187435, ..., -10.077262 ,
        -10.07667  , -10.0787945],
       [ -6.7151318,  -9.602398 ,  -9.5187435, ..., -10.077262 ,
        -10.07667  , -10.0787945],
       ...,
       [ -7.377645 ,  -9.713445 ,  -9.927865 , ..., -10.093688 ,
        -10.095995 , -10.098533 ],
       [ -7.377645 ,  -9.713445 ,  -9.927865 , ..., -10.093688 ,
        -10.095995 , -10.098533 ],
       [ -7.377645 ,  -9.713445 ,  -9.927865 , ..., -10.093688 ,
        -10.095995 , -10.098533 ]], dtype=float32), array([[ -6.7124968,  -9.547691 ,  -9.648446 , ..., -10.148347 ,
        -10.147022 , -10.147211 ],
       [ -6.7124968,  -9.547691 ,  -9.648446 , ..., -10.148347 ,
        -10.147022 , -10.147211 ],
       [ -6.7124968,  -9.547691 ,  -9.648446 , ..., -10.148347 ,
        -10.147022 , -10.147211 ],
       ...,
       [ -7.3926897,  -9.895513 , -10.14147  , ..., -10.213227 ,
        -10.218759 , -10.222772 ],
       [ -7.3926897,  -9.895513 , -10.14147  , ..., -10.213227 ,
        -10.218759 , -10.222772 ],
       [ -7.3926897,  -9.895513 , -10.14147  , ..., -10.213227 ,
        -10.218759 , -10.222772 ]], dtype=float32)), label_ids=(array([ 46,  46,  46, ..., 173, 173, 173]), array([ 46,  46,  46, ..., 173, 173, 173])), metrics={'test_loss': 1.5213046073913574, 'test_runtime': 1251.2588, 'test_samples_per_second': 27.753, 'test_steps_per_second': 3.469})
"""